<a href="https://colab.research.google.com/github/Charanmaloth/DS-Project-5-Webscraping/blob/main/Webscraping_Project4.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [1]:
!git clone 'https://github.com/Charanmaloth/DS-Project-5-Webscraping.git'

Cloning into 'DS-Project-5-Webscraping'...
remote: Enumerating objects: 3, done.
remote: Counting objects: 100% (3/3), done.
remote: Total 3 (delta 0), reused 0 (delta 0), pack-reused 0
Receiving objects: 100% (3/3), done.


In [2]:
!pip install geopandas

In [ ]:
from bs4 import BeautifulSoup
import requests

In [ ]:
response = requests.get('http://books.toscrape.com/')

In [ ]:
response.status_code

200

In [ ]:
print(response.text[:1000])

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html lang="en-us" class="no-js"> <!--<![endif]-->
    <head>
        <title>
    All products | Books to Scrape - Sandbox
</title>

        <meta http-equiv="content-type" content="text/html; charset=UTF-8" />
        <meta name="created" content="24th Jun 2016 09:29" />
        <meta name="description" content="" />
        <meta name="viewport" content="width=device-width" />
        <meta name="robots" content="NOARCHIVE,NOCACHE" />

        <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
        <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->

        
            <link rel="shortcut icon" href="static/oscar/favicon.

In [ ]:
type(response.text)

str

In [ ]:
html = response.text

In [ ]:
data = BeautifulSoup(html,'html.parser')

In [ ]:
data

<!DOCTYPE html>

<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!--> <html class="no-js" lang="en-us"> <!--<![endif]-->
<head>
<title>
    All products | Books to Scrape - Sandbox
</title>
<meta content="text/html; charset=utf-8" http-equiv="content-type"/>
<meta content="24th Jun 2016 09:29" name="created"/>
<meta content="" name="description"/>
<meta content="width=device-width" name="viewport"/>
<meta content="NOARCHIVE,NOCACHE" name="robots"/>
<!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
<!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
<link href="static/oscar/favicon.ico" rel="shortcut icon"/>
<link href="static/oscar/css/styles.css" rel="stylesheet" type="text/css"/>
<link href="s

In [ ]:
print(data.prettify())

<!DOCTYPE html>
<!--[if lt IE 7]>      <html lang="en-us" class="no-js lt-ie9 lt-ie8 lt-ie7"> <![endif]-->
<!--[if IE 7]>         <html lang="en-us" class="no-js lt-ie9 lt-ie8"> <![endif]-->
<!--[if IE 8]>         <html lang="en-us" class="no-js lt-ie9"> <![endif]-->
<!--[if gt IE 8]><!-->
<html class="no-js" lang="en-us">
 <!--<![endif]-->
 <head>
  <title>
   All products | Books to Scrape - Sandbox
  </title>
  <meta content="text/html; charset=utf-8" http-equiv="content-type"/>
  <meta content="24th Jun 2016 09:29" name="created"/>
  <meta content="" name="description"/>
  <meta content="width=device-width" name="viewport"/>
  <meta content="NOARCHIVE,NOCACHE" name="robots"/>
  <!-- Le HTML5 shim, for IE6-8 support of HTML elements -->
  <!--[if lt IE 9]>
        <script src="//html5shim.googlecode.com/svn/trunk/html5.js"></script>
        <![endif]-->
  <link href="static/oscar/favicon.ico" rel="shortcut icon"/>
  <link href="static/oscar/css/styles.css" rel="stylesheet" type="tex

In [ ]:
data.title
print(data.title.string)


    All products | Books to Scrape - Sandbox



In [ ]:
data.find('title').text

'\n    All products | Books to Scrape - Sandbox\n'

In [ ]:
data.find('title').text.strip()

'All products | Books to Scrape - Sandbox'

In [ ]:
# Function to convert star rating to a number
def convert_star_rating(rating_class):
  ratings = {
      'One' : 1,
      'Two' : 2,
      'Three' : 3,
      'Four' : 4,
      'Five' : 5
  }
  return ratings.get(rating_class, 0)


In [ ]:
#Function to Get Book Information from a Single Page
def get_books_info_from_page(data, base_url ):
  books_info = []
  books = data.find_all('article', class_='product_pod')

  for book in books:
    title = book.h3.a['title']
    rating = book.find('p', class_='star-rating')['class'][1]
    rating = convert_star_rating(rating)
    price = book.find('p',class_='price_color').text
    link = base_url + 'catalogue/' + book.h3.a['href']

    books_info.append({
        'title': title,
        'rating': rating,
        'price': price,
        'link': link
    })
  return books_info

In [ ]:
import pandas as pd

In [ ]:
#Base URL of the website
base_url = 'http://books.toscrape.com/'

# Initialize an empty list to store all book information
all_books_info = []
page_number = 1

while True:
    # Construct the URL for each page
    if page_number == 1:
        url = base_url
    else:
        url = f"{base_url}catalogue/page-{page_number}.html"
    response = requests.get(url)
    # Exit the loop if there are no more pages
    if response.status_code != 200:
        break
    html = response.text
    data = BeautifulSoup(html, 'html.parser')

    # Get book information from the current page
    books_info = get_books_info_from_page(data, base_url)
    # Exit the loop if there are no more books
    if not books_info:
        break
    # Add the current page's books to the all_books_info list
    all_books_info.extend(books_info)
    # Move to the next page
    page_number += 1

# Convert the list of dictionaries to a pandas DataFrame
books_df = pd.DataFrame(all_books_info)
# Print the DataFrame
print(books_df)

# Save the DataFrame to a CSV file
books_df.to_csv('books_info.csv', index=False)

# Display the first few rows of the DataFrame
print(books_df.head())

                                                 title  rating    price  \
0                                 A Light in the Attic       3  Â£51.77   
1                                   Tipping the Velvet       1  Â£53.74   
2                                           Soumission       1  Â£50.10   
3                                        Sharp Objects       4  Â£47.82   
4                Sapiens: A Brief History of Humankind       5  Â£54.23   
..                                                 ...     ...      ...   
995  Alice in Wonderland (Alice's Adventures in Won...       1  Â£55.53   
996   Ajin: Demi-Human, Volume 1 (Ajin: Demi-Human #1)       4  Â£57.06   
997  A Spy's Devotion (The Regency Spies of London #1)       5  Â£16.97   
998                1st to Die (Women's Murder Club #1)       1  Â£53.98   
999                 1,000 Places to See Before You Die       5  Â£26.08   

                                                  link  
0    http://books.toscrape.com/catalogue/c

In [ ]:
books_df.head()

,title,rating,price,link
0,A Light in the Attic,3,Â£51.77,http://books.toscrape.com/catalogue/catalogue/...
1,Tipping the Velvet,1,Â£53.74,http://books.toscrape.com/catalogue/catalogue/...
2,Soumission,1,Â£50.10,http://books.toscrape.com/catalogue/catalogue/...
3,Sharp Objects,4,Â£47.82,http://books.toscrape.com/catalogue/catalogue/...
4,Sapiens: A Brief History of Humankind,5,Â£54.23,http://books.toscrape.com/catalogue/catalogue/...


In [ ]:
books_df.shape

(1000, 4)